In [24]:
import sys
sys.path.append(r'c:\users\USERNAME\appdata\local\programs\python\python38-32\lib\site-packages')
%load_ext watermark
%watermark -d -u -a 'machine-maniac' -v -p numpy,scipy,matplotlib,sklearn

machine-maniac 
last updated: 2020-07-21 

CPython 3.7.4
IPython 7.8.0

numpy 1.16.5
scipy 1.3.1
matplotlib 3.1.1
sklearn 0.21.3


In [2]:
import numpy as np

In [3]:
#  1.1.1 1.1) Splitting a node (10 pts)
def split(array):
    frequency_dict = {}
    for i in range(0, len(array)):
        if array[i] in frequency_dict:
            frequency_dict[array[i]] = np.append(frequency_dict[array[i]], [i])
        else:
            frequency_dict[array[i]] = np.array([i])

    sorted_dict = {}
    for keys in sorted(frequency_dict.keys()):
        sorted_dict[keys] = frequency_dict[keys]
    return sorted_dict

print (split(np.array([0,1,2])))
print (split(np.array([1, 0, 1, 0, 0, 1, 0])))
print (split(np.array([1, 0, 3, 2, 0, 1, 1])))

{0: array([0]), 1: array([1]), 2: array([2])}
{0: array([1, 3, 4, 6]), 1: array([0, 2, 5])}
{0: array([1, 4]), 1: array([0, 5, 6]), 2: array([3]), 3: array([2])}


In [4]:
# 1.1.2 1.2) Implement Entropy (10 pts)
import math
def entropy(array):
    frequency_dict = split(array) 
    length_of_array = len(array)   
    frequency_array = []
    entropy = 0

    for i in range(0, len(frequency_dict)):
        frequency_array.append(len(list(frequency_dict.values())[i]))

    for i in range(0, len(frequency_array)):
        element_frequency = frequency_array[i]/length_of_array
        entropy = entropy + (element_frequency * math.log2(element_frequency))
    
    if entropy != 0:
        return entropy * -1

    return entropy

print(round(entropy(np.array([0, 1, 0, 1, 1, 0])), 4))
print(round(entropy(np.array([1, 2])), 4))
print(round(entropy(np.array([1, 1])), 4))
print(round(entropy(np.array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])), 4))
print(round(entropy(np.array([0, 0, 0])), 4))
print(round(entropy(np.array([1, 1, 1, 0, 1, 4, 4, 2, 1])), 4))

1.0
1.0
0.0
0.4395
0.0
1.6577


In [5]:
# 1.1.3 1.3) Implement Information Gain (10 pts)
def information_gain(x_array, y_array):
    parent_entropy = entropy(x_array)

    split_dict = split(y_array).values()
    
    for val in split_dict:
        freq = len(val)/len(x_array)
        child_entropy = entropy([x_array[i] for i in val])
        parent_entropy -= (child_entropy * freq)
        
    return parent_entropy


x = np.array([0, 1, 0, 1, 0, 1])
y = np.array([0, 1, 0, 1, 1, 1])
print(round(information_gain(x, y), 4)) #0.4591

x = np.array([0, 0, 1, 1, 2, 2])
y = np.array([0, 1, 0, 1, 1, 1])
print(round(information_gain(x, y), 4)) #0.2516


0.4591
0.2516


In [6]:
# 1.1.4 1.4) Decision Tree Splitting (10 pts)

def make_tree(X, y):
    # Return array if node is empty of pure (1 example on leaf node)
    if y.shape[0] == 1 or y.shape[0] == 0:
        return y
    # Compute information gain for each feature
    given_features_first = []
    given_features_second = []

    for i in range(0, X.shape[0]):
        given_features_first.append(X[i][0])
        given_features_second.append(X[i][1])
    
    given_features = np.array([given_features_first, given_features_second])

    # Compute the information gain for each of the features
    gains = []
    for i in range(0, X.shape[1]):
        gains.append(information_gain(given_features[i], y))

    # Early stopping if there is no information gain
    if (np.asarray(gains) <= 1e-05).all():
        return y

    # Else, get best feature
    best_feature = np.argmax(gains)

    results = {}

    # Use the split function to split on the best feature
    subset_dict = split(X[:, best_feature])

    # Note that each entry in the dictionary returned by
    # split is an attribute_value:array_indices pair.
    # here, we are going to iterate over these key-value
    # pairs and select the respective examples for the
    # new child nodes

    for feature_value, train_example_indices in subset_dict.items():
        child_y_subset = np.array([y[i] for i in train_example_indices])
        child_x_subset = np.array([X[i] for i in train_example_indices])

        # Next, we are using "recursion," that is, calling the same
        # tree_split function on the child subset(s)
        
        results["X_%d = %d" % (best_feature, feature_value)] = \
                make_tree(child_x_subset, child_y_subset)
    
    return results

x1 = np.array([0, 0, 1, 1, 2, 2])
x2 = np.array([0, 1, 0, 1, 0, 1])
X = np.array([x1, x2]).T
y = np.array([0, 1, 0, 1, 1, 1])
print('Inputs:\n', X)
print('\nLabels:\n', y)
print('\nDecision tree:\n', make_tree(X, y))


Inputs:
 [[0 0]
 [0 1]
 [1 0]
 [1 1]
 [2 0]
 [2 1]]

Labels:
 [0 1 0 1 1 1]

Decision tree:
 {'X_1 = 0': {'X_0 = 0': array([0]), 'X_0 = 1': array([0]), 'X_0 = 2': array([1])}, 'X_1 = 1': array([1, 1, 1])}


In [7]:

# 1.1.5 1.5) Building a Decision Tree API (10 pts)
import collections
from collections import Counter
class ID3DecisionTreeClassifer(object):
    def __init__(self):
        pass    
    def fit(self, X, y):
        self.splits_ = make_tree(X, y)
    def _majority_vote(self, label_array):
        counter = split(label_array)
        max_vote = list(counter.keys())[0] 
        return str(max_vote)
    def _traverse(self, x, d):
        if isinstance(d, np.ndarray):
            return d
        for key in d:
            name, value = key.split(' = ')
            feature_idx = int(name.split('_')[-1])
            value = int(value)
            if x[feature_idx] == value:
                return self._traverse(x, d[key])
    def predict(self, x):
        label_array = self._traverse(x, self.splits_)
        return self._majority_vote(label_array)

tree = ID3DecisionTreeClassifer()
tree.fit(X, y)
print(tree.predict(np.array([0, 0])))
print(tree.predict(np.array([0, 1])))
print(tree.predict(np.array([1, 0])))
print(tree.predict(np.array([1, 0])))
print(tree.predict(np.array([1, 1])))
print(tree.predict(np.array([2, 0])))
print(tree.predict(np.array([2, 1])))


0
1
0
0
1
1
1


In [8]:
# 1.2.1 2.1 Bootstrapping (10 pts)

from mlxtend.data import iris_data
X, y = iris_data()
print('Number of examples:', X.shape[0])
print('Number of features:', X.shape[1])
print('Unique class labels:', np.unique(y))

Number of examples: 150
Number of features: 4
Unique class labels: [0 1 2]


In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 45, train_size = 105, random_state = 123, stratify = y)
print('Number of training examples:', X_train.shape[0])
print('Number of test examples:', X_test.shape[0])

Number of training examples: 105
Number of test examples: 45


In [10]:
def draw_bootstrap_sample(rng, X, y):
    sample_indices = []
    for i in range (0, 105):
        sample_indices.append(i)
    bootstrap_indices = rng.choice(sample_indices, len(sample_indices))
    return X[bootstrap_indices], y[bootstrap_indices]

rng = np.random.RandomState(123)
X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
print('Number of training inputs from bootstrap round:', X_boot.shape[0])
print('Number of training labels from bootstrap round:', y_boot.shape[0])
print('Labels:\n', y_boot)

Number of training inputs from bootstrap round: 105
Number of training labels from bootstrap round: 105
Labels:
 [0 0 1 0 0 1 2 0 2 1 0 0 2 1 1 1 1 2 1 1 2 0 2 1 2 1 1 1 0 1 0 0 1 2 0 0 0
 0 2 1 1 2 1 2 1 1 2 1 2 0 1 1 2 2 1 0 1 0 2 2 0 1 0 2 0 0 0 0 1 2 0 0 1 0
 1 1 0 1 1 2 2 0 2 0 2 0 1 1 2 2 0 2 2 2 0 1 0 1 2 2 2 1 0 0 0]


In [11]:
# 1.2.2 2.2 Bagging classifier from decision trees

from sklearn.tree import DecisionTreeClassifier
class BaggingClassifier(object):
    def __init__(self, num_trees=10, random_state=123):
        self.num_trees = num_trees
        self.rng = np.random.RandomState(random_state)
    def fit(self, X, y):
        self.trees_ = [DecisionTreeClassifier(random_state=self.rng) for i in range(self.num_trees)]
        for i in range(self.num_trees):
            X_boot, y_boot = draw_bootstrap_sample(self.rng, X, y)
            self.trees_[i] = self.trees_[i].fit(X_boot, y_boot)
    def predict(self, X):   
        ary = np.zeros((X.shape[0], len(self.trees_)), dtype=np.int)
        for i in range(len(self.trees_)):
            ary[:, i] = self.trees_[i].predict(X)
            maj = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis=1, arr=ary)
        
        return maj

model = BaggingClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
print('Individual Tree Accuracies:')
for tree in model.trees_:
    predictions = tree.predict(X_test)
    print('%.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))
print('\nBagging Test Accuracy: %.1f%%' % ((predictions == y_test).sum() / X_test.shape[0] * 100))

Individual Tree Accuracies:
88.9%
93.3%
97.8%
93.3%
93.3%
93.3%
91.1%
97.8%
97.8%
97.8%

Bagging Test Accuracy: 97.8%


In [12]:
# 1.3.1 3.1 Bias-Variance decomposition of the 0-1 Loss for Decision Trees (10 pts)
rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype = np.int)
for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = DecisionTreeClassifier(random_state = 66).fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred

main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis = 0, arr = all_pred)

def calculate_bias(y_test, main_predictions):
    bias = 0
    for i in range(0, len(y_test)):
        if (y_test[i] != main_predictions[i]):
            bias = bias + y_test[i]
    return bias / len(y_test)

bias = calculate_bias(y_test, main_predictions)
print ('Average bias:', bias)

Average bias: 0.022222222222222223


In [13]:
def average_variance(all_pred, main_predictions):
    variance = 0
    for i in all_pred:
        variance = np.mean([(sum(i != main_predictions)/len(i))])
    return variance/2
    # return np.mean([(sum(preds!=main_predictions)/len(preds)) for preds in all_pred])

var = average_variance(all_pred, main_predictions)
print ('Average variance:', var)


Average variance: 0.03333333333333333


In [14]:
# 1.3.2 3.2 Bias-Variance decomposition of the 0-1 Loss for Bagging (10 pts)
from sklearn.ensemble import BaggingClassifier

rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype = np.int)

bagging_classifier = BaggingClassifier()

for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = bagging_classifier.fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred

main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis = 0, arr = all_pred)

bias = calculate_bias(y_test, main_predictions)
print ('Average bias:', bias)

var = average_variance(all_pred, main_predictions)
print ('Average variance:', var)

Average bias: 0.022222222222222223
Average variance: 0.022222222222222223


**Is the average variance higher or lower than the average of the decision tree in 3.1? And what about the average bias?** <br>
 The average variance is lower than as that in 3.1. Bagging helps to reduce variance since
 it's objective is the reduce the complexity of models that overfit the data.
 The average bias is the same as 3.1.

In [15]:
# 1.3.3 3.3 Bias-Variance decomposition of the 0-1 Loss for AdaBoost
from sklearn.ensemble import AdaBoostClassifier

rng = np.random.RandomState(123)
num_bootstrap = 200
all_pred = np.zeros((num_bootstrap, y_test.shape[0]), dtype = np.int)

ada_classifier = AdaBoostClassifier()

for i in range(num_bootstrap):
    X_boot, y_boot = draw_bootstrap_sample(rng, X_train, y_train)
    pred = ada_classifier.fit(X_boot, y_boot).predict(X_test)
    all_pred[i] = pred

main_predictions = np.apply_along_axis(lambda x: np.argmax(np.bincount(x)), axis = 0, arr = all_pred)

bias = calculate_bias(y_test, main_predictions)
print ('Average bias:', bias) # 0.06666666666666667

var = average_variance(all_pred, main_predictions)
print ('Average variance:', var) # 0.022222222222222223

Average bias: 0.06666666666666667
Average variance: 0.022222222222222223


**Is the average variance higher or lower than the average of the decision tree in 3.1? And what
about the average bias?** <br>
The average variance is lower than the variance in 3.1. AdaBoost is robust to over fitting. The
average bias is also higher. This may be because of the increase complexity of models that suffer from
high bias due to under fitting. 